# User Authentication with SQL in Python

This notebook demonstrates how to implement user signup and login functionality using SQLite in Python. We'll cover password hashing for security, user registration, and authentication.

## Import Required Libraries

We'll use sqlite3 for database operations and hashlib for secure password hashing.

In [1]:
import sqlite3
import hashlib
import os

## Connect to Database

We'll create a database for user authentication.

In [2]:
# Connect to the authentication database
conn = sqlite3.connect('auth.db')
cursor = conn.cursor()

print("Connected to authentication database successfully!")

Connected to authentication database successfully!


## Create Users Table

We'll create a table to store user information with hashed passwords for security.

In [3]:
# Create users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    username TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    email TEXT UNIQUE,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

print("Users table created successfully!")

Users table created successfully!


## Password Hashing Function

We'll create a function to securely hash passwords using SHA-256 with a salt.

In [4]:
def hash_password(password, salt=None):
    """Hash a password with a salt for security."""
    if salt is None:
        salt = os.urandom(32)  # Generate a random salt
        print(f"The random salt generated: {salt}")
    else:
        salt = salt.encode('utf-8') if isinstance(salt, str) else salt
        print(f"The random salt generated: {salt}")

    # Hash the password with the salt
    hashed = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)

    # Return salt + hash
    return salt + hashed

def verify_password(stored_hash, password):
    """Verify a password against its hash."""
    salt = stored_hash[:32]  # First 32 bytes are the salt
    expected_hash = hash_password(password, salt)
    return stored_hash == expected_hash

print("Password hashing functions defined!")

Password hashing functions defined!


## User Signup Function

Create a function to register new users with validation.

In [5]:
def signup(username, password, email):
    """Register a new user."""
    try:
        # Check if username already exists
        cursor.execute("SELECT user_id FROM users WHERE username = ?", (username,))
        if cursor.fetchone():
            return False, "Username already exists"

        # Check if email already exists
        cursor.execute("SELECT user_id FROM users WHERE email = ?", (email,))
        if cursor.fetchone():
            return False, "Email already exists"

        # Hash the password
        password_hash = hash_password(password)

        # Insert the new user
        cursor.execute("INSERT INTO users (username, password_hash, email) VALUES (?, ?, ?)",
                      (username, password_hash, email))
        conn.commit()

        return True, "User registered successfully"

    except sqlite3.Error as e:
        return False, f"Database error: {e}"

print("Signup function defined!")

Signup function defined!


## User Login Function

Create a function to authenticate existing users.

In [6]:
def login(username, password):
    """Authenticate a user."""
    try:
        # Get the user's stored hash
        cursor.execute("SELECT password_hash FROM users WHERE username = ?", (username,))
        result = cursor.fetchone()

        if not result:
            return False, "User not found"

        stored_hash = result[0]

        # Verify the password
        if verify_password(stored_hash, password):
            return True, "Login successful"
        else:
            return False, "Invalid password"

    except sqlite3.Error as e:
        return False, f"Database error: {e}"

print("Login function defined!")

Login function defined!


## Testing Signup and Login

Let's test the authentication functions with some example users.

In [7]:
# Test signup
print("Testing signup...")
success, message = signup("alice", "password123", "alice@example.com")
print(f"Signup result: {message}")

success, message = signup("bob", "securepass", "bob@example.com")
print(f"Signup result: {message}")

# Try to signup with existing username
success, message = signup("alice", "differentpass", "alice2@example.com")
print(f"Duplicate username result: {message}")

print("\nTesting login...")
# Test login
success, message = login("alice", "password123")
print(f"Login result for alice: {message}")

success, message = login("alice", "wrongpassword")
print(f"Wrong password result: {message}")

success, message = login("nonexistent", "password")
print(f"Non-existent user result: {message}")

success, message = login("bob", "securepass")
print(f"Login result for bob: {message}")

Testing signup...
The random salt generated: b"\xfa\xd6\xb9)\x14\x01\x88\xfa\x88Q'\x86\x91.\x17O\xe0\x87o\xe8\x17\x19\xf3#D\xd7\xbb\x92kEl\xdf"
Signup result: User registered successfully
The random salt generated: b'V\xc4,\xff\xa3BT\x81\xf3\xa5\xcd\xb8j\xc7\x8bH \xf6\xd1\x07h\x99\x18\x00\x1c\x9a\xce\xef\xc7@\xee*'
Signup result: User registered successfully
Duplicate username result: Username already exists

Testing login...
The random salt generated: b"\xfa\xd6\xb9)\x14\x01\x88\xfa\x88Q'\x86\x91.\x17O\xe0\x87o\xe8\x17\x19\xf3#D\xd7\xbb\x92kEl\xdf"
Login result for alice: Login successful
The random salt generated: b"\xfa\xd6\xb9)\x14\x01\x88\xfa\x88Q'\x86\x91.\x17O\xe0\x87o\xe8\x17\x19\xf3#D\xd7\xbb\x92kEl\xdf"
Wrong password result: Invalid password
Non-existent user result: User not found
The random salt generated: b'V\xc4,\xff\xa3BT\x81\xf3\xa5\xcd\xb8j\xc7\x8bH \xf6\xd1\x07h\x99\x18\x00\x1c\x9a\xce\xef\xc7@\xee*'
Login result for bob: Login successful
Signup result: User register

## Close Database Connection

Always close the database connection when done.

In [8]:
# Close the database connection
conn.close()

print("Database connection closed successfully!")

Database connection closed successfully!
